# **Anchor Google Drive file system**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Import libraries**

In [2]:
import os, re, argparse, json
from pathlib import Path
import numpy as np
import pandas as pd
import sys
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# **Global definitions**

In [3]:
input_dir = "/content/drive/MyDrive/AD_PM_ROAD/Input/"
output_dir = "/content/drive/MyDrive/AD_PM_ROAD/Output/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# **Load the Data**

In [4]:
def load_csv(path, drop_cols=None):
    df = pd.read_csv(path)
    if drop_cols:
        df = df.drop(drop_cols, axis=1)
    return df

data = {}
drop_cols = ['C2']  # columns to drop

# Helper to extract numeric parameter (accuracy) from key like "V_0.1"
def extract_acc(name):
    try:
        return float(name.split("_")[1])
    except:
        return 0.0  # fallback

for rep_number in os.listdir(input_dir):
    data[rep_number] = {}

    rep_path = os.path.join(input_dir, rep_number)
    # sort keys numerically by <acc>
    keys = sorted(os.listdir(rep_path), key=extract_acc)

    for key in keys:
        anomaly_type, parameter = key.split("_")

        data[rep_number][key] = {"Training": [], "Test": []}

        # -------------------
        # Training data
        # -------------------
        training_dir = os.path.join(rep_path, key, "TrainingTimeSeries")
        if os.path.exists(training_dir):
            training_files = sorted([os.path.join(training_dir, f) for f in os.listdir(training_dir)])
            with ThreadPoolExecutor() as executor:
                data[rep_number][key]["Training"] = list(executor.map(lambda p: load_csv(p, drop_cols), training_files))

        # -------------------
        # Test data (positive windows)
        # -------------------
        test_dir = os.path.join(rep_path, key, "TestTimeSeries")
        pos_test = []
        if os.path.exists(test_dir):
            test_files = sorted([os.path.join(test_dir, f) for f in os.listdir(test_dir)])
            with ThreadPoolExecutor() as executor:
                pos_test = list(executor.map(lambda p: load_csv(p, drop_cols), test_files))
        data[rep_number][key]["Test"].extend([[df, 1] for df in pos_test])

        # -------------------
        # Test data (negative windows from other anomaly type)
        # -------------------
        other_type = "W" if anomaly_type == "V" else "V"
        other_test_dir = os.path.join(rep_path, f"{other_type}_{parameter}", "TestTimeSeries")
        neg_test = []
        if os.path.exists(other_test_dir):
            other_test_files = sorted([os.path.join(other_test_dir, f) for f in os.listdir(other_test_dir)])
            with ThreadPoolExecutor() as executor:
                neg_test = list(executor.map(lambda p: load_csv(p, drop_cols), other_test_files))
        data[rep_number][key]["Test"].extend([[df, 0] for df in neg_test])

        print(f"{rep_number} | {key} -> Training: {len(data[rep_number][key]['Training'])}, Test: {len(data[rep_number][key]['Test'])}")

1 | V_0.1 -> Training: 75, Test: 48
1 | W_0.1 -> Training: 35, Test: 48
1 | V_0.2 -> Training: 72, Test: 48
1 | W_0.2 -> Training: 33, Test: 48
1 | V_0.3 -> Training: 69, Test: 48
1 | W_0.3 -> Training: 31, Test: 48
1 | V_0.4 -> Training: 66, Test: 48
1 | W_0.4 -> Training: 31, Test: 48
1 | V_0.5 -> Training: 63, Test: 48
1 | W_0.5 -> Training: 29, Test: 48
1 | V_0.6 -> Training: 59, Test: 48
1 | W_0.6 -> Training: 27, Test: 48
1 | V_0.7 -> Training: 56, Test: 48
1 | W_0.7 -> Training: 27, Test: 48
1 | V_0.8 -> Training: 53, Test: 48
1 | W_0.8 -> Training: 25, Test: 48
1 | V_0.9 -> Training: 50, Test: 48
1 | W_0.9 -> Training: 23, Test: 48
1 | V_1.0 -> Training: 48, Test: 48
1 | W_1.0 -> Training: 23, Test: 48
2 | V_0.1 -> Training: 75, Test: 48
2 | W_0.1 -> Training: 35, Test: 48
2 | V_0.2 -> Training: 72, Test: 48
2 | W_0.2 -> Training: 33, Test: 48
2 | V_0.3 -> Training: 69, Test: 48
2 | W_0.3 -> Training: 31, Test: 48
2 | V_0.4 -> Training: 66, Test: 48
2 | W_0.4 -> Training: 31, T

# **Train/test**

In [9]:
# Custom RMSE loss
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true) ** 2) + 1e-8)

criterion_rmse = RMSELoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------
# Dataset class
# --------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, X):
        self.X = X
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx]

# --------------------------
# Autoencoder classes
# --------------------------
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1):
        super().__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        enc_out, _ = self.encoder(x)
        dec_out, _ = self.decoder(enc_out)
        return dec_out.permute(0, 2, 1)

class BiLSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1):
        super().__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.decoder = nn.LSTM(2*hidden_dim, input_dim, num_layers, batch_first=True)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        enc_out, _ = self.encoder(x)
        dec_out, _ = self.decoder(enc_out)
        return dec_out.permute(0, 2, 1)

class GRUAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1):
        super().__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.decoder = nn.GRU(hidden_dim, input_dim, num_layers, batch_first=True)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        enc_out, _ = self.encoder(x)
        dec_out, _ = self.decoder(enc_out)
        return dec_out.permute(0, 2, 1)

# --------------------------
# Training/evaluation function
# --------------------------
def train_one_class_ae(model_class, X_train_tensor, X_test_tensor, y_test, name, epochs=50):
    train_ds = TimeSeriesDataset(X_train_tensor)
    train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)

    model = model_class(input_dim=X_train_tensor.shape[1]).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for xb in train_dl:
            xb = xb.to(device)
            opt.zero_grad()
            recon = model(xb)
            loss = criterion_rmse(recon, xb)
            loss.backward()
            opt.step()
            total_loss += loss.item() * xb.size(0)
        total_loss /= len(train_dl.dataset)

    model.eval()
    with torch.no_grad():
        recon_test = model(X_test_tensor.to(device))
        errors = torch.mean((recon_test - X_test_tensor.to(device))**2, dim=[1,2]).cpu().numpy()

        recon_train = model(X_train_tensor.to(device))
        train_errors = torch.mean((recon_train - X_train_tensor.to(device))**2, dim=[1,2]).cpu().numpy()

    threshold = np.mean(train_errors) + np.std(train_errors)
    y_pred = (errors > threshold).astype(int)
    y_pred_corrected = 1 - y_pred
    f1 = f1_score(y_test, y_pred_corrected)
    return f1

# --------------------------
# Pad/truncate helper
# --------------------------
def pad_truncate(arrays, target_len):
    res = []
    for x in arrays:
        T, C = x.shape
        if T < target_len:
            pad = np.zeros((target_len - T, C), dtype=np.float32)
            x_new = np.vstack([x, pad])
        else:
            x_new = x[:target_len, :]
        res.append(x_new)
    return np.stack(res)

# --------------------------
# Main evaluation loop
# --------------------------
results = {}

for rep_number in sorted(data.keys()):
    print(f"\nProcessing repetition: {rep_number}")
    results.setdefault(rep_number, {})

    for key in sorted(data[rep_number].keys()):
        print(f"\nDataset key: {key}")
        training_time_series = data[rep_number][key]["Training"]
        test_time_series = data[rep_number][key]["Test"]

        # Prepare train/test arrays
        X_train = [df.values.astype(np.float32) for df in training_time_series]
        X_test = [df.values.astype(np.float32) for df, _ in test_time_series]
        y_test = np.array([label for _, label in test_time_series], dtype=int)

        # Determine uniform sequence length
        all_lengths = [x.shape[0] for x in X_train + X_test]
        seq_len = int(np.median(all_lengths))

        # Pad/truncate
        X_train = pad_truncate(X_train, seq_len)
        X_test = pad_truncate(X_test, seq_len)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(np.transpose(X_train, (0, 2, 1)), dtype=torch.float32)
        X_test_tensor = torch.tensor(np.transpose(X_test, (0, 2, 1)), dtype=torch.float32)

        # Train and evaluate models
        f1_scores = {}
        for model_name, model_class in [("LSTM", LSTMAutoencoder),
                                        ("BiLSTM", BiLSTMAutoencoder),
                                        ("GRU", GRUAutoencoder)]:
            f1 = train_one_class_ae(model_class, X_train_tensor, X_test_tensor, y_test, model_name)
            print(f"  {model_name} F1: {f1:.4f}")
            f1_scores.setdefault(model_name, []).append(f1)

        # Store results per repetition
        results[rep_number][key] = f1_scores

# --------------------------
# Compute mean ± std per key across repetitions and save
# --------------------------
with open(output_dir + "f1_results.txt", "w") as f:
    for key in sorted({k for rep in results.values() for k in rep.keys()}):
        f.write(f"{key}:\n")
        print(f"\nSummary for {key}:")
        for model_name in ["LSTM", "BiLSTM", "GRU"]:
            scores = [results[rep][key][model_name][0] for rep in results if key in results[rep]]
            mean_f1 = np.mean(scores)
            std_f1 = np.std(scores)
            f.write(f"{model_name}: {mean_f1:.4f} ± {std_f1:.4f}\n")
            print(f"  {model_name}: {mean_f1:.4f} ± {std_f1:.4f}")


Processing repetition: 1

Dataset key: V_0.1
  LSTM F1: 0.0588
  BiLSTM F1: 0.1667
  GRU F1: 0.6383

Dataset key: V_0.2
  LSTM F1: 0.4255
  BiLSTM F1: 0.4000
  GRU F1: 0.2703

Dataset key: V_0.3
  LSTM F1: 0.6038
  BiLSTM F1: 0.6087
  GRU F1: 0.6939

Dataset key: V_0.4
  LSTM F1: 0.6429
  BiLSTM F1: 0.7451
  GRU F1: 0.4167

Dataset key: V_0.5
  LSTM F1: 0.8000
  BiLSTM F1: 0.6939
  GRU F1: 0.8148

Dataset key: V_0.6
  LSTM F1: 0.8358
  BiLSTM F1: 0.7451
  GRU F1: 0.7778

Dataset key: V_0.7
  LSTM F1: 0.8406
  BiLSTM F1: 0.8852
  GRU F1: 0.8615

Dataset key: V_0.8
  LSTM F1: 0.8696
  BiLSTM F1: 0.8070
  GRU F1: 0.9062

Dataset key: V_0.9
  LSTM F1: 0.9538
  BiLSTM F1: 0.8421
  GRU F1: 0.8667

Dataset key: V_1.0
  LSTM F1: 0.9677
  BiLSTM F1: 0.9355
  GRU F1: 0.9508

Dataset key: W_0.1
  LSTM F1: 0.3214
  BiLSTM F1: 0.2909
  GRU F1: 0.1633

Dataset key: W_0.2
  LSTM F1: 0.4262
  BiLSTM F1: 0.4667
  GRU F1: 0.3214

Dataset key: W_0.3
  LSTM F1: 0.5085
  BiLSTM F1: 0.3509
  GRU F1: 0.3729